In [1]:
import pandas as pd
import numpy as np
import folium
import json

In [2]:
# !pip install folium

In [3]:
f = open('geojson_lnglat.json', 'r')
geojson = json.load(f)

In [4]:
locations = ['Rembrandtpark',
 'Sarphatipark',
 'Vondelpark',
 'Oosterpark',
 'Erasmuspark',
 'Westerpark']

visits = [10, 20, 40, 60, 70, 90]

data = pd.DataFrame(list(zip(locations, visits)),
               columns =['Location', 'Visitors'])
data

,Location,Visitors
0,Rembrandtpark,10
1,Sarphatipark,20
2,Vondelpark,40
3,Oosterpark,60
4,Erasmuspark,70
5,Westerpark,90


In [5]:
filtered = [geo for geo in geojson['features'] if geo['properties']['Naam'] in locations]

In [6]:
geojson['features'] = filtered
# geojson

In [7]:
style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.0, 
                            'weight': 0.5}
highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.5, 
                                'weight': 0.5}

m = folium.Map(
    location=[52.377956, 4.897070],
    tiles="cartodbpositron",
    zoom_start=13,
)

chor = folium.Choropleth(
    geo_data=geojson,
    name="choropleth",
    data=data,
    columns=["Location", "Visitors"],
    key_on="properties.Naam",
    bins=[0, 25, 50, 75, 100],
    fill_color="YlOrRd",
    fill_opacity=0.8,
    line_opacity=0.7,
    legend_name="Visitors",
    highlight=True,
)

chor.add_to(m)

gjson = folium.GeoJson(geojson, 
                       name="City Parks",
                       style_function=style_function,
                       highlight_function=highlight_function,
                      ).add_to(m)

folium.features.GeoJsonPopup(fields=['Naam'], labels=False).add_to(gjson)

In [8]:
m